In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [4]:
importlib.reload(ATools)    
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data("tdc_only.pkl", trigger_channel=4, bcr_channel=2, amount_of_events=784_480)


Reading: 100%|█████████▉| 784479/784480 [01:20<00:00, 9730.90Events/s] 


 ...]

In [7]:
#Putting the data into the files
buffer = analyser.anubis_data.copy()
interval = 100_000
with tqdm(total=len(buffer), desc=f"Chunking", unit='BCR') as pbar:
    while buffer:
        storage = f"data/bcr/bcr{pbar.n}.pkl"
        with open(storage, "wb") as f:
            pickle.dump(buffer[:interval], f)
        buffer = buffer[interval:]
        pbar.update(interval)


Chunking: 31600000BCR [04:51, 108552.12BCR/s]                             


In [8]:
import gc
importlib.reload(ATools)

def correctionBCR(bcrs):
    gc.disable()
    last = 0
    current = 0
    bad_stage = False
    period = 1_048_575
    interval = 114_048
    while current < len(bcrs):
        now_bcr = bcrs[current]
        if now_bcr.error and not bad_stage:
            bad_stage = True
            last = current - 1
            good_bcr = bcrs[last]
            triggers = good_bcr.triggers.copy() #handle triggers later
            good_bcr.triggers = []
        
        if bad_stage:
            delta = now_bcr.hitTime - good_bcr.hitTime
            if delta < 0:
                delta += period
            if  delta % interval < 50 or interval - (delta % interval) < 50: #no overflow
                prob_bcr_count  = round(delta / interval)
                obs_bcr_count = current - last
                for bad_bcr in range(last+1, current): #remove and replace
                    del bcrs[last+1]
                for new_bcr in range(0,prob_bcr_count):
                    if new_bcr != 0:
                        bcrs.insert(last+new_bcr, AtTools.BCR((good_bcr.hitTime+(new_bcr)*interval) % period, good_bcr.event_time))
                    for trig in triggers:
                        previous = (good_bcr.hitTime+(new_bcr)*interval) % period
                        next = (good_bcr.hitTime+(new_bcr+1)*interval) % period
                        overflow = False
                        if previous > next:
                            next += period
                            overflow = True
                        if previous < trig.hitTime < next or (overflow and previous < trig.hitTime + period < next):
                            bcrs[last+new_bcr].add_trigger(trig)
                            triggers.remove(trig)
                        else:
                            break
                now_bcr.error = False
                bad_stage = False
                current += prob_bcr_count-obs_bcr_count
            else:
                triggers += now_bcr.triggers
                if current - last > 9:
                    bad_stage = False
                    bcrs[current+1].error = False

            if not now_bcr.error and bad_stage:
                bad_stage = False
        current += 1
    return bcrs

In [9]:
data_list = sorted([f for f in os.listdir("data//bcr") if os.path.isfile(os.path.join("data//bcr", f))], key=lambda x: int(x[3:-4]))
print(data_list)
with tqdm(total=len(data_list), desc=f"Chunking", unit='files') as pbar:
    for data in data_list:
        with open(f"data//bcr//{data}", "rb") as f:
            bcrs = pickle.load(f)
        bcrs = correctionBCR(bcrs)
        with open(f"data//bcr//{data}", "wb") as f:
            pickle.dump(bcrs, f)
        pbar.update(1)
        #gc.collect()

['bcr0.pkl', 'bcr100000.pkl', 'bcr200000.pkl', 'bcr300000.pkl', 'bcr400000.pkl', 'bcr500000.pkl', 'bcr600000.pkl', 'bcr700000.pkl', 'bcr800000.pkl', 'bcr900000.pkl', 'bcr1000000.pkl', 'bcr1100000.pkl', 'bcr1200000.pkl', 'bcr1300000.pkl', 'bcr1400000.pkl', 'bcr1500000.pkl', 'bcr1600000.pkl', 'bcr1700000.pkl', 'bcr1800000.pkl', 'bcr1900000.pkl', 'bcr2000000.pkl', 'bcr2100000.pkl', 'bcr2200000.pkl', 'bcr2300000.pkl', 'bcr2400000.pkl', 'bcr2500000.pkl', 'bcr2600000.pkl', 'bcr2700000.pkl', 'bcr2800000.pkl', 'bcr2900000.pkl', 'bcr3000000.pkl', 'bcr3100000.pkl', 'bcr3200000.pkl', 'bcr3300000.pkl', 'bcr3400000.pkl', 'bcr3500000.pkl', 'bcr3600000.pkl', 'bcr3700000.pkl', 'bcr3800000.pkl', 'bcr3900000.pkl', 'bcr4000000.pkl', 'bcr4100000.pkl', 'bcr4200000.pkl', 'bcr4300000.pkl', 'bcr4400000.pkl', 'bcr4500000.pkl', 'bcr4600000.pkl', 'bcr4700000.pkl', 'bcr4800000.pkl', 'bcr4900000.pkl', 'bcr5000000.pkl', 'bcr5100000.pkl', 'bcr5200000.pkl', 'bcr5300000.pkl', 'bcr5400000.pkl', 'bcr5500000.pkl', 'bcr56

Chunking: 100%|██████████| 316/316 [05:09<00:00,  1.02BCR/s]


In [13]:
#connect the data
data_list = sorted([f for f in os.listdir("data//bcr") if os.path.isfile(os.path.join("data//reset", f))])
bcrs = []
for data in data_list:
    with open(f"data//bcr//{data}", "rb") as f:
        bcrs += pickle.load(f)
print("done")
with open("data//bcr_corrected.pkl", "wb") as f:
    pickle.dump(bcrs, f)

done


In [14]:
with open("data//bcr_corrected.pkl", "rb") as f:
    bcrs = pickle.load(f)
print(sum([bcr.error*1 for bcr in bcrs])/len(bcrs))

0.0015962436678549211
